In [1]:
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import Float, Thread
from mitsuba.core.xml import load_file
from mitsuba.python.util import traverse
from mitsuba.python.autodiff import render, write_bitmap
from mitsuba.core import Color3f

# Load the Cornell Box
Thread.thread().file_resolver().append('C:/Users/Max/Code/mitsuba2/resources/data/scenes/spot/')
# scene = load_file('C:/Users/Max/Code/mitsuba2/resources/data/scenes/spot/spot_sens.xml')
scene = load_file('C:/Users/Max/Code/mitsuba2/resources/data/scenes/spot/spot_int.xml')

# Find differentiable scene parameters
params = traverse(scene)

print(params)

2021-11-03 12:08:47 INFO main [optix_api.cpp:56] Dynamic loading of the Optix library ..

2021-11-03 12:08:47 INFO main [xml.cpp:1221] Loading XML file "C:\Users\Max\Code\mitsuba2\resources\data\scenes\spot\spot_int.xml" ..

2021-11-03 12:08:47 INFO main [xml.cpp:1222] Using variant "gpu_autodiff_rgb"

2021-11-03 12:08:47 INFO main [xml.cpp:355] "C:\Users\Max\Code\mitsuba2\resources\data\scenes\spot\spot_int.xml": in-memory version upgrade (v2.0.0 -> v2.2.1) ..

2021-11-03 12:08:47 INFO main [PluginManager] Loading plugin "plugins\conductor.dll" ..

...

2021-11-03 12:08:47 INFO main [PluginManager] Loading plugin "plugins\gaussian.dll" ..

2021-11-03 12:08:47 INFO main [PluginManager] Loading plugin "plugins\tex_samp.dll" ..

2021-11-03 12:08:47 INFO main [PluginManager] Loading plugin "plugins\independent.dll" ..

2021-11-03 12:08:47 INFO main [Scene] Building scene in OptiX ..

ParameterMap[
    my_envmap.scale,
  * my_envmap.data,
    my_envmap.resolution,
  * bsdf.specular_reflectance.value,
  * bsdf.eta.value,
  * bsdf.k.value,
    spot.to_world,
    spot.vertex_count,
    spot.face_count,
  * spot.faces_buf,
  * spot.vertex_positions_buf,
  * spot.vertex_normals_buf,
  * spot.vertex_texcoords_buf,
    TexSampCamera.near_clip,
    TexSampCamera.far_clip,
    TexSampCamera.focus_distance,
    TexSampCamera.shutter_open,
    TexSampCamera.shutter_open_time,
]


In [ ]:
# Make a backup copy
param_res = params['my_envmap.resolution']
param_ref = Float(params['my_envmap.data'])

# Discard all parameters except for one we want to differentiate
params.keep(['my_envmap.data'])

# Render a reference image (no derivatives used yet)
image_ref = render(scene, spp=16)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_ref.png', image_ref, crop_size)

Error: Session cannot generate requests

![temp/out_ref.png](temp/out_ref.png)

In [3]:
# Change to a uniform white lighting environment
params['my_envmap.data'] = ek.full(Float, 1.0, len(param_ref))
params.update()

from mitsuba.python.autodiff import Adam
opt = Adam(params, lr=.02)

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_start.png', image, crop_size)
write_bitmap('temp/envmap_start.png', params['my_envmap.data'],
                 (param_res[0], param_res[1]))

for it in range(100):
    # Perform a differentiable rendering of the scene
    image = render(scene, optimizer=opt, unbiased=True, spp=1)

    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step
    opt.step()

    err_ref = ek.hsum(ek.sqr(param_ref - params['my_envmap.data']))
    print('Iteration %03i: error=%g' % (it, err_ref[0]))


image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_end.png', image, crop_size)
write_bitmap('temp/envmap_end.png', params['my_envmap.data'],
                 (param_res[0], param_res[1]))


Iteration 000: error=85684.1
Iteration 001: error=82818.4
Iteration 002: error=80000.7
Iteration 003: error=77235.4
Iteration 004: error=74526.2
Iteration 005: error=71875.1
Iteration 006: error=69284.6
Iteration 007: error=66756.1
Iteration 008: error=64289.6
Iteration 009: error=61885.4
Iteration 010: error=59544.2
Iteration 011: error=57266.6
Iteration 012: error=55053.1
Iteration 013: error=52903.7
Iteration 014: error=50818.6
Iteration 015: error=48797.4
Iteration 016: error=46840.2
Iteration 017: error=44946.7
Iteration 018: error=43116.7
Iteration 019: error=41349.6
Iteration 020: error=39644.4
Iteration 021: error=38000.8
Iteration 022: error=36418.4
Iteration 023: error=34895.6
Iteration 024: error=33432
Iteration 025: error=32027.1
Iteration 026: error=30679.4
Iteration 027: error=29387.6
Iteration 028: error=28150.9
Iteration 029: error=26968
Iteration 030: error=25837.3
Iteration 031: error=24757.9
Iteration 032: error=23728.4
Iteration 033: error=22747.3
Iteration 034: err

![start](temp/out_start.png)
![end](temp/out_end.png)
![env_start](temp/envmap_start.png)
![env_end](temp/envmap_end.png)